<a href="https://colab.research.google.com/github/gatihe/datascience_studies/blob/master/Aplicac%CC%A7a%CC%83o_do_Algoritmo_A_para_visitar_pontos_turi%CC%81sticos_na_Ita%CC%81lia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplicação do Algoritmo A* para planejamento de visita à pontos turísticos na Itália

## 1. Introdução:
### 1.1 Motivação:

Este projeto tem como objetivo apresentar as características do algoritmo A* (A-Estrela) e propor a sua utilização para a resolução de um problema simples.

### 1.2 Seleção do problema e estados objetivos:

Domínio do problema: Identificar a melhor rota entre os três pontos turisticos da Itália mais bem avaliados pelo site TripAdvisor.

De acordo com o TripAdvisor (plataforma digital reconhecida como o maior site de viagens do mundo), os pontos turísticos mais bem avaliados são:
- Sassi di Matera (Província de Taranto)
- St Peter's Basílica (Roma)
- Gardaland Park (Verona)

(inserir imagem da busca do tripadvisor)

Sendo assim, a solução proposta deve apresentar o caminho menos custoso à ser percorrido para um turista que pretenda visitar estes três pontos.

Considerando o contexto de agentes de busca, é possível dizer que os três destinos podem ser considerados os estados objetivos do problema, desde que ao fim da execução do algoritmo, todos tenham sido visitados. Enquanto, para fins práticos o ponto de partida será considerado o local de destino que apresenta o menor preço de passagem aérea entre São Paulo (GRU) e Itália. No dia 10 de junho de 2020, o vôo mais barato encontrado na plataforma Google Flights foi com destino à Venice (Veneza), que passa à ser então o ponto de partida do algoritmo (estado inicial do problema).

(inserir imagem do google flights)


## 2. Metodologia:
### 2.1 Estrutura do agente:

O agente de busca neste problema pode ser considerado o algoritmo de busca A* que é implementado pela função ```astar_search()```, definida como:

In [0]:
start_node_value = 0
def astar_search(graph, start, end, start_node_value):
    
    heuristics = set_heuristics(end)

    # Create lists for open nodes and closed nodes
    open = []
    closed = []
    opened_list = []
    closed_list = []
    opened_list_entry = []
    closed_list_entry = []
    contador_de_execucao = 0
    next_start_node_value = 0
    nodes_values = []

    # Create a start node and an goal node
    start_node = Node(start, None)
    goal_node = Node(end, None)

    # Add the start node
    open.append(start_node)
    #To save last execution g value
    start_node.g = start_node.g + start_node_value
    # Loop until the open list is empty
    while len(open) > 0:
        opened_list_entry = []
        closed_list_entry = []

        # Sort the open list to get the node with the lowest cost first
        open.sort()

        # Get the node with the lowest cost
        current_node = open.pop(0)

        # Add the current node to the closed list
        closed.append(current_node)
        closed_list_entry.append(current_node.name)
        closed_list.append(closed_list_entry)
        
        # Check if we have reached the goal, return the path
        if current_node == goal_node:
            path = []
            while current_node != start_node:
                path.append(current_node.name + ': ' + str(current_node.g))
                nodes_values.append(current_node.g)
                current_node = current_node.parent
            path.append(start_node.name + ': ' + str(start_node.g))
            # Return reversed path
            next_start_node_value = nodes_values[0]
            return path[::-1], opened_list, closed_list, next_start_node_value

        # Get neighbours
        neighbors = graph.get(current_node.name)
        
        # Loop neighbors
        for key, value in neighbors.items():

            # Create a neighbor node
            neighbor = Node(key, current_node)

            # Check if the neighbor is in the closed list
            if(neighbor in closed):
                continue

            # Calculate full path cost

            neighbor.g = current_node.g + graph.get(current_node.name, neighbor.name)
            neighbor.h = heuristics.get(neighbor.name)
            neighbor.f = neighbor.g + neighbor.h
            # Check if neighbor is in open list and if it has a lower f value
            if(add_to_open(open, neighbor) == True):
                # Everything is green, add neighbor to open list
                open.append(neighbor)
                opened_list_entry.append(neighbor.name)
        opened_list.append(opened_list_entry)

    # Return None, no path is found
    return None

In [0]:
# Check if a neighbor should be added to open list
def add_to_open(open, neighbor):
    for node in open:
        if (neighbor == node and neighbor.f > node.f):
            return False
    return True

def print_results(path,opened_list, closed_list):
    print('Caminho (Nó:Custo acumulado):')
    print(path)
    print("\n")
    #Printing A* executions/open/closed nodes
    opened_list.pop()
    print('Lista de novos nós abertos:')
    print(opened_list)
    print("\n")
    print('Lista de novos nós fechados:')
    print(closed_list)
    return

def gather_paths(path, new_path_piece):
  for i in new_path_piece:
    path.append(i)
  return path

### 2.2 Estrutura de dados:

In [0]:
# This class represent a graph
class Graph:

    # Initialize the class
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    # Create an undirected graph by adding symmetric edges
    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist

    # Add a link from A and B of given distance, and also add the inverse link if the graph is undirected
    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance

    # Get neighbors or a neighbor
    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    # Return a list of nodes in the graph
    def nodes(self):
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)

In [0]:
# This class represent a node
class Node:

    # Initialize the class
    def __init__(self, name:str, parent:str):
        self.name = name
        self.parent = parent
        self.g = 0 # Distance to start node
        self.h = 0 # Distance to goal node
        self.f = 0 # Total cost

    # Compare nodes
    def __eq__(self, other):
        return self.name == other.name

    # Sort nodes
    def __lt__(self, other):
         return self.f < other.f

### 2.3 Função heurística adotada:
#### 2.3.1 Distâncias em linha reta

In [5]:
import pandas as pd
import numpy as np
tabela_heuristicas = pd.read_csv("distancias_estacoes_italia.csv", index_col = 0)
tabela_heuristicas

,Bolzano,Verona,Milano,Torino,Trieste,Venice,Bologna,Genoa,Pisa,Florence,Ancona,Rome,Pescara,Naples,Foggia,Bari,Brindisi,Lecce,Taranto,Regio di Calabria
Bolzano,0,124,209,325,207,139,223,297,315,305,361,518,501,668,651,744,832,878,821,995
Verona,124,0,138,263,218,98,107,199,197,185,285,411,421,576,575,676,778,813,748,901
Milano,209,138,0,131,356,243,200,120,218,249,399,480,524,659,677,786,892,925,857,975
Torino,325,263,131,0,480,365,296,125,264,318,490,525,600,713,753,865,971,1004,931,1022
Trieste,207,218,356,480,0,114,228,404,342,288,227,429,358,535,487,563,653,690,640,853
Venice,139,98,243,365,114,0,130,290,246,204,224,393,366,534,513,605,701,739,680,861
Bologna,223,107,200,296,228,130,0,192,116,84,199,304,325,471,480,587,690,725,657,797
Genoa,297,199,120,125,404,290,192,0,141,199,377,403,480,592,631,744,849,884,810,900
Pisa,315,197,218,264,342,246,116,141,0,69,249,264,340,450,489,604,709,742,668,764
Florence,305,185,249,318,288,204,84,199,69,0,181,232,281,409,435,546,652,686,613,729


Além da obtenção e organização dos dados referentes às distâncias em linha reta de uma estação de trem à outra, a quantidade de caminhos possíveis exige que a função referente ao agente de busca seja executada diversas vezes. Para facilitar a manipulação do agente de busca, foi implementada a seguinte função, que tem como objetivo buscar, de forma automática, na tabela de distâncias em linha reta o valor das funções heurísticas de acordo com o destino escolhido.

In [0]:
stations_list = list(tabela_heuristicas.columns.values)
heuristics_dict = tabela_heuristicas.to_dict()
heuristics = {}

def set_heuristics(station_name):
    station_index = stations_list.index(station_name)
    heuristics = [value for value in heuristics_dict.values()][station_index]
    return heuristics

#### 2.3.2 Disposição das estações e o custo aproximado das rotas de trem na Itália



![Italy_Train_Routes_Costs](https://raw.githubusercontent.com/gatihe/si702-astar/master/img/cost-map-italy.png?token=AKCYUZDXJXQGDPHKYRXZA6S64CLIQ)

## Mapa de estações

![Italy_Train_Routes_Costs](https://raw.githubusercontent.com/gatihe/si702-astar/master/img/italy-train-map.png?token=AKCYUZEZHWUB55TUKRZCDOK64CLLA)




#### 2.3.3 Grafo gerado a partir do custo aproximado e mapa de estações

In [0]:
# Create a graph
graph = Graph()

# Create graph connections (Actual distance)
graph.connect('Bolzano', 'Verona', 152)
graph.connect('Torino', 'Milano', 144)
graph.connect('Torino', 'Genoa', 172)
graph.connect('Milano', 'Genoa', 150)
graph.connect('Milano', 'Bologna', 219)
graph.connect('Milano', 'Verona', 163)
graph.connect('Verona', 'Venice', 120)
graph.connect('Verona', 'Bologna', 149)
graph.connect('Venice', 'Trieste', 157)
graph.connect('Venice', 'Bologna', 153)
graph.connect('Bologna', 'Florence', 118)
graph.connect('Bologna', 'Ancona', 220)
graph.connect('Pisa', 'Genoa', 164)
graph.connect('Pisa', 'Rome', 354)
graph.connect('Florence', 'Rome', 278)
graph.connect('Rome', 'Ancona', 300)
graph.connect('Rome', 'Pescara', 219)
graph.connect('Rome', 'Naples', 223)
graph.connect('Ancona', 'Pescara', 161)
graph.connect('Naples', 'Taranto', 305)
graph.connect('Naples', 'Regio di Calabria', 491)
graph.connect('Bari', 'Brindisi', 114)
graph.connect('Bari', 'Taranto', 85)
graph.connect('Taranto', 'Regio di Calabria', 385)
graph.connect('Brindisi', 'Lecce', 40)

# Make graph undirected, create symmetric connections
graph.make_undirected()

## 3. Resultados

Considerando o estado inicial do problema e os possíveis estados finais, constata-se que o melhor caminho está contido entre as seguintes possibilidades:


![Teste](https://raw.githubusercontent.com/gatihe/si702-astar/master/img/Untitled%20Diagram.png?token=AKCYUZDDBVI3JGHFX5IQA3S64CNMC)

- **Hipótese 1:** Venice -> ... -> Rome -> ... -> Taranto -> ... -> Verona
- **Hipótese 2:** Venice -> ... -> Rome -> ... -> Verona -> ... -> Taranto
- **Hipótese 3:** Venice -> ... -> Taranto -> ... -> Verona -> ... -> Rome
- **Hipótese 4:** Venice -> ... -> Taranto -> ... -> Rome -> ... -> Verona
- **Hipótese 5:** Venice -> ... -> Verona -> ... -> Taranto -> ... -> Rome
- **Hipótese 6:** Venice -> ... -> Verona -> ... -> Rome -> ... -> Taranto
---

### Hipótese 1:

*Venice -> ... -> Rome -> ... -> Taranto -> ... -> Verona*

In [108]:
#Venice to Rome
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)



Caminho
['Venice: 0', 'Bologna: 153', 'Florence: 271', 'Rome: 549', 'Naples: 772', 'Taranto: 1077', 'Naples: 1382', 'Rome: 1605', 'Florence: 1883', 'Bologna: 2001', 'Verona: 2150']
Lista de abertos:
[[['Trieste', 'Bologna', 'Verona'], ['Florence', 'Ancona', 'Milano'], ['Rome'], ['Bolzano', 'Milano']], [['Ancona', 'Pescara', 'Naples', 'Pisa', 'Florence'], ['Taranto', 'Regio di Calabria']], [['Regio di Calabria', 'Naples', 'Bari'], ['Brindisi'], ['Rome'], ['Ancona', 'Pescara', 'Pisa', 'Florence'], ['Lecce'], ['Bologna'], ['Milano', 'Verona', 'Venice'], []]]
Lista de fechados:
[[['Venice'], ['Bologna'], ['Florence'], ['Verona'], ['Rome']], [['Rome'], ['Naples'], ['Taranto']], [['Taranto'], ['Bari'], ['Naples'], ['Rome'], ['Brindisi'], ['Florence'], ['Bologna'], ['Lecce'], ['Verona']]]
Custo total:
2150


#### Custo total: 2150


---



### Hipótese 2:

*Venice -> ... -> Rome -> ... -> Verona -> ... -> Taranto*

In [109]:
#Venice to Rome
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)



Caminho
['Venice: 0', 'Bologna: 153', 'Florence: 271', 'Rome: 549', 'Florence: 827', 'Bologna: 945', 'Verona: 1094', 'Bologna: 1243', 'Florence: 1361', 'Rome: 1639', 'Naples: 1862', 'Taranto: 2167']
Lista de abertos:
[[['Trieste', 'Bologna', 'Verona'], ['Florence', 'Ancona', 'Milano'], ['Rome'], ['Bolzano', 'Milano']], [['Ancona', 'Pescara', 'Naples', 'Pisa', 'Florence'], ['Bologna'], ['Milano', 'Verona', 'Venice']], [['Venice', 'Bologna', 'Bolzano', 'Milano'], ['Trieste'], ['Florence', 'Ancona'], ['Pescara', 'Rome'], [], ['Rome'], [], ['Naples', 'Pisa'], [], ['Genoa', 'Torino'], ['Taranto', 'Regio di Calabria']]]
Lista de fechados:
[[['Venice'], ['Bologna'], ['Florence'], ['Verona'], ['Rome']], [['Rome'], ['Florence'], ['Bologna'], ['Verona']], [['Verona'], ['Venice'], ['Bologna'], ['Ancona'], ['Pescara'], ['Florence'], ['Trieste'], ['Rome'], ['Bolzano'], ['Milano'], ['Naples'], ['Taranto']]]
Custo total:
2167


#### Custo total: 2167


---



### Hipótese 3:

*Venice -> ... -> Taranto -> ... -> Verona -> ... -> Rome*

In [110]:
#Venice to Rome
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)



Caminho
['Venice: 0', 'Bologna: 153', 'Florence: 271', 'Rome: 549', 'Naples: 772', 'Taranto: 1077', 'Naples: 1382', 'Rome: 1605', 'Florence: 1883', 'Bologna: 2001', 'Verona: 2150', 'Bologna: 2299', 'Florence: 2417', 'Rome: 2695']
Lista de abertos:
[[['Trieste', 'Bologna', 'Verona'], [], ['Florence', 'Ancona', 'Milano'], ['Pescara', 'Rome'], [], ['Bolzano', 'Milano'], ['Rome'], ['Naples', 'Pisa'], ['Taranto', 'Regio di Calabria']], [['Regio di Calabria', 'Naples', 'Bari'], ['Brindisi'], ['Rome'], ['Ancona', 'Pescara', 'Pisa', 'Florence'], ['Lecce'], ['Bologna'], ['Milano', 'Verona', 'Venice'], []], [['Venice', 'Bologna', 'Bolzano', 'Milano'], ['Florence', 'Ancona'], ['Rome'], ['Trieste']]]
Lista de fechados:
[[['Venice'], ['Trieste'], ['Bologna'], ['Ancona'], ['Pescara'], ['Verona'], ['Florence'], ['Rome'], ['Naples'], ['Taranto']], [['Taranto'], ['Bari'], ['Naples'], ['Rome'], ['Brindisi'], ['Florence'], ['Bologna'], ['Lecce'], ['Verona']], [['Verona'], ['Bologna'], ['Florence'], ['Ven

#### Custo total: 2695


---



### Hipótese 4:

*Venice -> ... -> Taranto -> ... -> Rome -> ... -> Verona*

In [111]:
#Venice to Taranto
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)



Caminho
['Venice: 0', 'Bologna: 153', 'Florence: 271', 'Rome: 549', 'Naples: 772', 'Taranto: 1077', 'Naples: 1382', 'Rome: 1605', 'Florence: 1883', 'Bologna: 2001', 'Verona: 2150']
Lista de abertos:
[[['Trieste', 'Bologna', 'Verona'], [], ['Florence', 'Ancona', 'Milano'], ['Pescara', 'Rome'], [], ['Bolzano', 'Milano'], ['Rome'], ['Naples', 'Pisa'], ['Taranto', 'Regio di Calabria']], [['Regio di Calabria', 'Naples', 'Bari'], ['Brindisi'], ['Rome']], [['Ancona', 'Pescara', 'Naples', 'Pisa', 'Florence'], ['Bologna'], ['Milano', 'Verona', 'Venice']]]
Lista de fechados:
[[['Venice'], ['Trieste'], ['Bologna'], ['Ancona'], ['Pescara'], ['Verona'], ['Florence'], ['Rome'], ['Naples'], ['Taranto']], [['Taranto'], ['Bari'], ['Naples'], ['Rome']], [['Rome'], ['Florence'], ['Bologna'], ['Verona']]]
Custo total:
2150


#### Custo total: 2150


---



### Hipótese 5:

*Venice -> ... -> Verona -> ... -> Taranto -> ... -> Rome*

In [112]:
#Venice to Taranto
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)



Caminho
['Venice: 0', 'Verona: 120', 'Bologna: 269', 'Florence: 387', 'Rome: 665', 'Naples: 888', 'Taranto: 1193', 'Naples: 1498', 'Rome: 1721']
Lista de abertos:
[[['Trieste', 'Bologna', 'Verona']], [['Venice', 'Bologna', 'Bolzano', 'Milano'], ['Trieste'], ['Florence', 'Ancona'], ['Pescara', 'Rome'], [], ['Rome'], [], ['Naples', 'Pisa'], [], ['Genoa', 'Torino'], ['Taranto', 'Regio di Calabria']], [['Regio di Calabria', 'Naples', 'Bari'], ['Brindisi'], ['Rome']]]
Lista de fechados:
[[['Venice'], ['Verona']], [['Verona'], ['Venice'], ['Bologna'], ['Ancona'], ['Pescara'], ['Florence'], ['Trieste'], ['Rome'], ['Bolzano'], ['Milano'], ['Naples'], ['Taranto']], [['Taranto'], ['Bari'], ['Naples'], ['Rome']]]
Custo total:
1721


#### Custo total: 1721


---



### Hipótese 6:

*Venice -> ... -> Verona -> ... -> Rome -> ... -> Taranto*

In [113]:
#Venice to Taranto
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Verona', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Rome', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Taranto', start_node_value)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)



Caminho
['Venice: 0', 'Verona: 120', 'Bologna: 269', 'Florence: 387', 'Rome: 665', 'Naples: 888', 'Taranto: 1193']
Lista de abertos:
[[['Trieste', 'Bologna', 'Verona']], [['Venice', 'Bologna', 'Bolzano', 'Milano'], ['Florence', 'Ancona'], ['Rome'], ['Trieste']], [['Ancona', 'Pescara', 'Naples', 'Pisa', 'Florence'], ['Taranto', 'Regio di Calabria']]]
Lista de fechados:
[[['Venice'], ['Verona']], [['Verona'], ['Bologna'], ['Florence'], ['Venice'], ['Rome']], [['Rome'], ['Naples'], ['Taranto']]]
Custo total:
1193


#### Custo total: 1193


---

